In [2]:
import sys
import os
from pathlib import Path
import json
import pandas as pd

# Root Dataset

In [3]:
root_path = r"C:\Users\gabe7\Downloads\Fundus_Dataset"

**Move all video paths into one list**

In [4]:
# Seperate training videos from labels
training_dirs = []
for dir in os.listdir(root_path):
    dir_path = os.path.join(root_path, dir)
    training_dirs.append(dir_path)

# Label directory is first item
label_dir = training_dirs.pop(0)

In [5]:
training_videos = []
for video_dir in training_dirs:
    # Each directory has 1 more directory to enter then videos
    inner_dir = os.listdir(video_dir)[0]
    video_dir_path = os.path.join(video_dir, inner_dir)
    for video_file in os.listdir(video_dir_path):
        # Full video file path
        video_path = os.path.join(video_dir_path, video_file)
        # Aggregate all video paths into 1 list
        training_videos.append(video_path)

**Labels**
- Want peak and trough mask per SVP present video
- Start and end frames of pulsations, may be multiple per video
- Images of peak and trough frames

In [6]:
# JSON Files
disease_path = r"C:\Users\gabe7\Downloads\Fundus_Dataset\Labels\video_information\disease.json"
disease_variety_path = r"C:\Users\gabe7\Downloads\Fundus_Dataset\Labels\video_information\disease_variety.json"
temporal_localization_path = r"C:\Users\gabe7\Downloads\Fundus_Dataset\Labels\Temporal\Temporal_Localization\Temporal_localization.json"
peak_trough_metadata_path = r"C:\Users\gabe7\Downloads\Fundus_Dataset\Labels\Temporal\Peak_and_trough\Metadata.json"
training_path = r"C:\Users\gabe7\Downloads\Fundus_Dataset\Labels\Spatial\metadata\training.json"
validation_path = r"C:\Users\gabe7\Downloads\Fundus_Dataset\Labels\Spatial\metadata\validation.json"
testing_path = r"C:\Users\gabe7\Downloads\Fundus_Dataset\Labels\Spatial\metadata\testing.json"

In [7]:
# Image and mask files
peak_trough_image_dir = r"C:\Users\gabe7\Downloads\Fundus_Dataset\Labels\Temporal\Peak_and_trough\images"
peak_trough_mask_dir = r"C:\Users\gabe7\Downloads\Fundus_Dataset\Labels\Temporal\Peak_and_trough\masks"

In [23]:
# Load training JSON
with open(training_path, 'r') as f:
    train_dict = json.load(f)

In [24]:
# Load validation JSON
with open(validation_path, 'r') as f:
    val_dict = json.load(f)

In [25]:
# Load testing JSON
with open(testing_path, 'r') as f:
    test_dict = json.load(f)

In [26]:
# Load metadata JSON
with open(peak_trough_metadata_path, 'r') as f:
    peak_trough_metadata = json.load(f)

In [27]:
len(peak_trough_metadata)

335

# Train Validation Test Split

In [30]:
# Retrieve ID's from pre split
training_ids = train_dict.keys()
validation_ids = val_dict.keys()
testing_ids = test_dict.keys()

In [54]:
# Retrieve each training sample only if there is ID match
training_samples = []
for id, value in peak_trough_metadata.items():
    if id in training_ids:
        training_samples.append(value)
len(training_samples)

254

In [55]:
# Retrieve each validation sample only if there is ID match
validation_samples = []
for id, value in peak_trough_metadata.items():
    if id in validation_ids:
        validation_samples.append(value)
len(validation_samples)

13

In [53]:
# Retrieve each testing sample only if there is ID match
testing_samples = []
for id, value in peak_trough_metadata.items():
    if id in testing_ids:
        testing_samples.append(value)
len(testing_samples)

68